# 🚘 Used Car Pricing Model

**Goal:** The goal of this project is to estimate the resale price of a used Ford Focus using regression.

## Preprocessing

### Import and Read Data

The first step is to import necessary libraries and read the raw data into a Pandas DataFrame.

In [ ]:
import pandas as pd

data = pd.read_csv('focus.csv')
data

,model,year,price,transmission,mileage,fuelType,engineSize
0,Focus,2016,8000,Manual,38852,Petrol,1.0
1,Focus,2019,13400,Manual,11952,Petrol,1.0
2,Focus,2019,14600,Manual,22142,Petrol,1.5
3,Focus,2016,9450,Manual,14549,Diesel,1.6
4,Focus,2015,9999,Manual,7010,Diesel,1.6
...,...,...,...,...,...,...,...
5449,Focus,2019,18745,Manual,7855,Diesel,2.0
5450,Focus,2019,16350,Manual,13891,Petrol,1.0
5451,Focus,2019,16850,Manual,13452,Petrol,1.0
5452,Focus,2019,17310,Automatic,13376,Petrol,1.0


In [ ]:
data['transmission'].unique()

array(['Manual', 'Automatic', 'Semi-Auto'], dtype=object)

In [ ]:
data['fuelType'].unique()

array(['Petrol', 'Diesel'], dtype=object)

There are 6 input features: `year`, `transmission`, `mileage`, `fuelType`, and `engineSize`. The `model` column can be dropped because it is constant across each row. The `price` column is the output ($y$) variable.

### Convert Categorical Data to Dummy Values

Before running regression, the *categorical (nominal) data* must first be converted to *dummy values*. Dummy values indicate the absence or presence of a feature with a `0` or `1` value. For nominal data with $k$ possible values, we create $k-1$ dummy variables. The `drop_first=True` parameter drops the first of the $k$ possible values, leading to $k-1$ features. The two nominal features that must be converted are `transmission` and `fuelType`.

In [ ]:
transmission = pd.get_dummies(data['transmission'], drop_first=True)
fuelType = pd.get_dummies(data['fuelType'], drop_first=True)

In [ ]:
transmission.head()

,Manual,Semi-Auto
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [ ]:
fuelType.head()

,Petrol
0,1
1,1
2,1
3,0
4,0


### Organize the DataFrame

The following step joins dummy variables, drops unnecessary features, and rearranges features in the DataFrame.

In [ ]:
data_processed = data.drop(columns=['model', 'transmission', 'fuelType'])
data_processed = data_processed.join(transmission)
data_processed = data_processed.join(fuelType)
data_processed = data_processed.rename(columns={'engineSize': 'engine_size', 'Manual': 'manual', 'Semi-Auto': 'semi_auto', 'Petrol': 'petrol'})
data_processed

,year,price,mileage,engine_size,manual,semi_auto,petrol
0,2016,8000,38852,1.0,1,0,1
1,2019,13400,11952,1.0,1,0,1
2,2019,14600,22142,1.5,1,0,1
3,2016,9450,14549,1.6,1,0,0
4,2015,9999,7010,1.6,1,0,0
...,...,...,...,...,...,...,...
5449,2019,18745,7855,2.0,1,0,0
5450,2019,16350,13891,1.0,1,0,1
5451,2019,16850,13452,1.0,1,0,1
5452,2019,17310,13376,1.0,0,0,1


### Split into Train/Test Data

The data must be split into a *training set* to build the model and a *test set* to evaluate its effectiveness. Here, 75% of the data is used to train and 25% is used to test.

In [ ]:
from sklearn.model_selection import train_test_split

X = data_processed.drop(columns='price')
y = data_processed['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
X.loc[0]

year            2016.0
mileage        38852.0
engine_size        1.0
manual             1.0
semi_auto          0.0
petrol             1.0
Name: 0, dtype: float64

In [ ]:
y.loc[0]

8000

**Challenge**: Without looking at `X_train`, `X_test`, `y_train`, or `y_test`, determine the shapes of these arrays. Hint: consider the test size and the shape of `data`.

## Model 1: Univariate Linear Regression

The first model will use just one numeric variable: `mileage`, to predict the output variable: `price`.

### Visualize the Shape of the Data

In order to develop an intuition around which function will fit the data best, it is helpful to visualize the relationship between $x$ (`mileage`) and $y$ (`price`) in the training data.

In [ ]:
X_train_mileage = X_train['mileage']
X_test_mileage = X_test['mileage']

In [ ]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

In [ ]:
train_plot = figure(title='Training Data', x_axis_label='Mileage', y_axis_label='Price')
train_plot.circle(x=X_train_mileage, y=y_train)
show(train_plot)

The data is likely either cubic or close to the function $\frac{1}{\log(x)}$. We will try models that give those types of functions soon, but for now let's try a linear model.

**Challenge**: See what happens when we don't reshape our mileage data using `reshape(-1, 1)`. Try to interpret the error message.

### Building a Linear Model

The entire training step for linear regression is as follows. SKLearn does a lot of the hard work for us!

In [ ]:
from sklearn import linear_model

reg = linear_model.LinearRegression()
reg.fit(X_test_mileage.to_frame(), y_test)

LinearRegression()

Now that our model is trained we can see what hypothesis function we have built. In SKLearn, `model.coef_` tells us the value of $\theta_1$, and `model.intercept_` tells us $\theta_0$.

In [ ]:
print(reg.intercept_)
print(reg.coef_)

17613.26080726479
[-0.1772997]


So our hypothesis function is approximately:

$$
\text{Price} = 17470 + -0.169 \star \text{Mileage}
$$

note that you might have slightly different values, but they should be in about the same range.

This function gives us, in mathematical terms, a prediction for the price of any car given its mileage.

We can calculate these predictions for many cars at once using `predict`.

In [ ]:
train_predictions_linear = reg.predict(X_train_mileage.to_frame())
test_predictions_linear = reg.predict(X_test_mileage.to_frame())

### Evaluating the Model

Now let's evaluate our model to see how well we're doing in predicting the actual car prices.

In [ ]:
from bokeh.layouts import row

def visualize_model(X_train, X_test, train_preds, test_preds):
    train_plot_cubic = figure(title='Train Data', x_axis_label='Mileage', y_axis_label='Price')
    test_plot_cubic = figure(title='Test Data', x_axis_label='Mileage', y_axis_label='Price')

    train_plot_cubic.circle(x=X_train.ravel(), y=y_train, color='blue', legend_label='Actual')
    train_plot_cubic.circle(x=X_train.ravel(), y=train_preds, color='red', legend_label='Predicted')

    test_plot_cubic.circle(x=X_test.ravel(), y=y_test, color='blue', legend_label='Actual')
    test_plot_cubic.circle(x=X_test.ravel(), y=test_preds, color='red', legend_label='Predicted')

    show(row(train_plot_cubic,test_plot_cubic))

visualize_model(X_train_mileage, X_test_mileage, train_predictions_linear, test_predictions_linear)

test_score = reg.score(X_test_mileage.to_frame(), y_test)
train_score = reg.score(X_train_mileage.to_frame(), y_train)
print(f'Test score: {test_score}')
print(f'Train score: {train_score}')

Test score: 0.5482934538236803
Train score: 0.5492394341496594


**Challenge**: Go back and fit the model using the test data instead, then rerun the cells up to this point. Are there differences in the scores and/or graphs? Try to explain any differences or any lack of differences.


## Model 2: Univariate Polynomial Regression

The first model will use just one numeric variable: `mileage`, to predict the output variable: `price`.

### Cubic Model

The next model is cubic. That means it takes the form $f(x) = \theta_0 + \theta_1 x + \theta_2 x^2 + \theta_3 x^3$ for constants $\theta_0, \theta_1, \theta_2, \theta_3$, where $x$ is the `mileage` variable and $f(x)$ is the predicted price.

Note that $\theta_0$ is referred to as the *bias term*, or the *intercept term*. It is not affected by the input variable, and its purpose is to allow the algorithm to find the *intercept* of the model.


First, the single feature $x$, is transformed into multiple features $x^1, x^2, x^3$.

Note: by default, `PolynomialFeatures` includes the bias term, but `LinearRegression` does this automatically, so we will not include it here.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

cubic = PolynomialFeatures(degree=3, include_bias=False)
X_train_mileage_cubic = cubic.fit_transform(X_train_mileage.to_frame())
X_test_mileage_cubic = cubic.fit_transform(X_test_mileage.to_frame())
print(pd.DataFrame(X_train_mileage_cubic))

            0             1             2
0      7777.0  6.048173e+07  4.703664e+11
1      5491.0  3.015108e+07  1.655596e+11
2      9439.0  8.909472e+07  8.409651e+11
3     53130.0  2.822797e+09  1.499752e+14
4     10223.0  1.045097e+08  1.068403e+12
...       ...           ...           ...
4085  17127.0  2.933341e+08  5.023934e+12
4086  18680.0  3.489424e+08  6.518244e+12
4087   3000.0  9.000000e+06  2.700000e+10
4088  11979.0  1.434964e+08  1.718944e+12
4089  15811.0  2.499877e+08  3.952556e+12

[4090 rows x 3 columns]


Then, linear regression is run to fit the model and generate predictions for the train and test set.


In [ ]:
cubic_model = linear_model.LinearRegression()
cubic_model.fit(X_test_mileage_cubic, y_test)

LinearRegression()

Once more, we can determine what function the algorithm has generated.


In [ ]:
print(cubic_model.intercept_)
print(cubic_model.coef_)

20213.043040375927
[-4.81096203e-01  6.81558998e-06 -3.73690217e-11]



**Challenge**: Try to write out the mathematical function of our model using the above outputs. You can round the numbers if it makes it easier.

Now we can make our predictions:

In [ ]:
train_predictions_cubic = cubic_model.predict(X_train_mileage_cubic)
test_predictions_cubic = cubic_model.predict(X_test_mileage_cubic)

Finally, predicted results are plotted against the train and test data and assigned an $R^2$ value (measure of how close the data fit the model).

In [ ]:
visualize_model(X_train_mileage, X_test_mileage, train_predictions_cubic, test_predictions_cubic)
print('Train Data R-squared: ' + str(cubic_model.score(X_train_mileage_cubic,y_train)))
print('Test Data R-squared: ' + str(cubic_model.score(X_test_mileage_cubic,y_test)))

Train Data R-squared: 0.5377700500000632
Test Data R-squared: 0.6087464621695298


**Challenge**: Go back and fit the cubic model using the test data instead, then rerun the cells up to this point. Are there differences in the scores and/or graphs? Try to explain any differences or any lack of differences. Also compare the differences to what we saw previously with the linear model.

### Reciprocal Logarithmic Model

The next approach is a model similar in shape to $\frac{1}{\log(x)}$.

First, the data is transformed to match the reciprocal $\log$ function $\frac{1}{\log(x)}$.

In [ ]:
import numpy as np

def addReciprocalLogFeatures(numeric):
    log_feats = numeric.copy()
    valid = (log_feats != 1) & (log_feats > 0)
    log_feats[valid] = np.log(log_feats[valid]) / np.log(10)
    log_feats[log_feats <= 0] = 1e-10
    rec_log_feats = 1 / log_feats
    return np.hstack([numeric, rec_log_feats, numeric * rec_log_feats])

X_train_mileage_rl = addReciprocalLogFeatures(X_train_mileage.to_frame())
X_test_mileage_rl = addReciprocalLogFeatures(X_test_mileage.to_frame())

print('X_train_mileage_rl \n' + str(X_train_mileage_rl[:4,:]))

X_train_mileage_rl 
[[7.77700000e+03 2.57015753e-01 1.99881151e+03]
 [5.49100000e+03 2.67404600e-01 1.46831866e+03]
 [9.43900000e+03 2.51577011e-01 2.37463541e+03]
 [5.31300000e+04 2.11624992e-01 1.12436358e+04]]


Once again, linear regression is run to fit the model and generate predictions for the train and test set.

In [ ]:
rl_model = linear_model.LinearRegression()
rl_model.fit(X_train_mileage_rl, y_train)

train_predictions_rl = rl_model.predict(X_train_mileage_rl)
test_predictions_rl = rl_model.predict(X_test_mileage_rl)

And again, predicted results are plotted against the train and test data and assigned an $R^2$ value (measure of how close the data fit the model).

In [ ]:
visualize_model(X_train_mileage, X_test_mileage, train_predictions_rl, test_predictions_rl)

print(f'Train data R-squared: {rl_model.score(X_train_mileage_rl, y_train)}')
print(f'Test data R-squared: {rl_model.score(X_test_mileage_rl, y_test)}')

Train data R-squared: 0.609824278735335
Test data R-squared: 0.6076988061590105


Now we have tried two different polynomial models, but neither is scoring very high. To help increase our score, let's now look at how we can provide our model with more information so that it can make better predictions.

**Challenge**: Try to think of a reason why a reciprocal logarithmic model might be preferable for our dataset. Hint: you can go to [Desmos](https://www.desmos.com/calculator) and enter $\frac{1}{\log{x}}$ to see the graph of the function.

In [ ]:
# Price can never be zero, so we want our model to never predict zero, so the mathematical function should not give zero

## Principle Component Analysis (PCA)

*Principal Component Analysis* flattens multiple numeric features into fewer features (known as components) that preserve the most important information from the original data. This allows for easy visualization of the data to get an intuition of which function would be best applied.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

num_cols = ['year', 'mileage', 'engine_size']
X_train_numeric = X_train[num_cols]
X_test_numeric = X_test[num_cols]

scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.fit_transform(X_test_numeric)

pca = PCA(n_components=1)
X_train_pca = pca.fit_transform(X_train_numeric)

train_plot_pca = figure(title='PCA on Numeric Inputs vs Price', x_axis_label='Principal Component', y_axis_label='Price')
train_plot_pca.circle(x=X_train_pca.ravel(), y=y_train)
show(train_plot_pca)

The above plot suggests that both cubic and reciprocal logarithmic functions might be good fits for the data.

**Challenge**: Is there any difference in the graph if we don't scale our data first?

## Model 3: Multivariate Polynomial Regression

The final model will use all numeric variables: `year`, `mileage`, `engine_size`, and dummy variables `manual`, `semi_auto`, `petrol`, to predict the output variable: `price`.

### Cubic Model
The first model is a cubic one. That means it takes the form $y = ax^3 + bx^2 + cx + d$ for constants $a, b, c, d$. This time however, every variable is used as input, and not just `mileage`.

First, each numeric feature is transformed into polynomial features of degree 3.

In [ ]:
X_train_numeric_cubic = cubic.fit_transform(X_train_numeric)
X_test_numeric_cubic = cubic.fit_transform(X_test_numeric)
print(X_train_numeric_cubic.shape)

(4090, 19)


Before running regression to fit the model, the features are scaled to make the orders of magnitude roughly the same. They are then combined with the ordinal features from before.

In [ ]:
X_train_cubic_numeric_scaled = scaler.fit_transform(X_train_numeric_cubic)
X_test_cubic_numeric_scaled = scaler.fit_transform(X_test_numeric_cubic)

nominal_cols = ['petrol', 'semi_auto', 'manual']
X_train_nominal = X_train[nominal_cols]
X_test_nominal = X_test[nominal_cols]



# We can sub out the scaled version of the data here 
X_train_cubic_full = np.hstack([X_train_numeric_cubic, X_train_nominal])
X_test_cubic_full = np.hstack([X_test_numeric_cubic, X_test_nominal])

X_train_cubic_full.shape

(4090, 22)

Finally, the cubic model is fit using `RidgeCV` - an advanced form of linear regression that *regularizes* data to prevent overfitting.

In [ ]:
cubic_model = linear_model.RidgeCV()
cubic_model.fit(X_train_cubic_full, y_train)

RidgeCV()

In [ ]:
train_predictions_cubic = cubic_model.predict(X_train_cubic_full)
test_predictions_cubic = cubic_model.predict(X_test_cubic_full)
test_predictions_cubic

array([-2.68764034e+20, -2.65075293e+20, -1.41736178e+20, ...,
       -2.37091919e+20, -2.65522209e+20, -2.68642011e+20])

In [ ]:
cubic_model.score(X_train_cubic_full, y_train)

-3.9972998439487753e+34

Closeness of fit is assessed using $R^2$ once again.

In [ ]:
cubic_model.score(X_test_cubic_full, y_test)

-1.7649898804017265e+34

Instead of plotting the predictions, the following code compares the predicted price to the actual price, and calculates, on average, how far off the prediction was (for the test data).

In [ ]:
df = pd.DataFrame({"Predicted": test_predictions_cubic, "Actual": y_test})
df['% Difference'] = (abs(df['Predicted']-df['Actual'])/df['Actual'])*100

print("Percentage Difference between Predicted and Actual Values (Cubic Model)")
print(df.head())
print("\nMean % Difference between Predicted and Actual Values: " + str(df['% Difference'].mean()) +"%")

Percentage Difference between Predicted and Actual Values (Cubic Model)
         Predicted  Actual  % Difference
3049 -2.687640e+20   30999  8.670087e+17
2093 -2.650753e+20   11999  2.209145e+18
3481 -1.417362e+20    8000  1.771702e+18
1900 -2.564593e+20   14000  1.831852e+18
3913 -2.654821e+20   12900  2.058001e+18

Mean % Difference between Predicted and Actual Values: 4.755628224730617e+18%


### Reciprocal Logarithmic Model

Another approach is a model similar in shape to $\frac{1}{\log(x)}$, with all the features.

First, the numeric features are transformed to match the reciprocal $\log$ function $\frac{1}{\log(x)}$.

In [ ]:
X_train_rl = addReciprocalLogFeatures(X_train_numeric)
X_test_rl = addReciprocalLogFeatures(X_test_numeric)
X_train_rl.shape

(4090, 9)

Before running regression to fit the model, the features are scaled to make the orders of magnitude roughly the same. They are then combined with the ordinal features from before.

In [ ]:
X_train_rl_scaled = scaler.fit_transform(X_train_rl)
X_test_rl_scaled = scaler.fit_transform(X_test_rl)


# We can sub out the scaled version of the data here 
X_train_rl_full = np.hstack([X_train_rl, X_train_nominal])
X_test_rl_full = np.hstack([X_test_rl, X_test_nominal])

X_train_rl_full.shape

(4090, 12)

Again, the reciprocal $\log$ model is fit using `RidgeCV` - a *regularized* version of regression.

In [ ]:
rl_model = linear_model.RidgeCV()
rl_model.fit(X_train_rl_full, y_train)

RidgeCV()

In [ ]:
train_predictions_rl = rl_model.predict(X_train_rl_full)
test_predictions_rl = rl_model.predict(X_test_rl_full)
test_predictions_rl

array([-6162182.49671579, -6171283.13236764, -6175503.80965945, ...,
       -6173240.35990044, -6168747.01068826, -6166720.78423326])

In [ ]:
rl_model.score(X_train_rl_full, y_train)

-984350664.0859494

Once again, closeness of fit is measured with $R^2$.

In [ ]:
rl_model.score(X_test_rl_full, y_test)

-2196670746.776959

Instead of plotting the predictions, the following code compares the predicted price to the actual price, and calculates, on average, how far off the prediction was (for the test data).

In [ ]:
df = pd.DataFrame({"Predicted": test_predictions_rl, "Actual": y_test})
df['% Difference'] = (abs(df['Predicted']-df['Actual'])/df['Actual'])*100

print("Percentage Difference between Predicted and Actual Values (Reciprocal Log Model)")
print(df.head())
print("\nMean % Difference between Predicted and Actual Values: " + str(df['% Difference'].mean()) +"%")

Percentage Difference between Predicted and Actual Values (Reciprocal Log Model)
         Predicted  Actual  % Difference
3049 -6.162182e+06   30999  19978.649301
2093 -6.171283e+06   11999  51531.645407
3481 -6.175504e+06    8000  77293.797621
1900 -6.170364e+06   14000  44174.025640
3913 -6.169983e+06   12900  47929.325909

Mean % Difference between Predicted and Actual Values: 159282.79661946298%


**Challenge**: Try retraining and evaluating the above two models, but this time without scaling the data. What are the differences?